In [1]:
import torch
import numpy as np

### Tensor基本操作

In [39]:
x = torch.arange(12, dtype = torch.float32).reshape((3,4))
y = torch.zeros(12,dtype = torch.float32).reshape((3,4))
x, y

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]))

In [42]:
torch.cat((x, y), dim=0), torch.cat((x, y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]]),
 tensor([[ 0.,  1.,  2.,  3.,  0.,  0.,  0.,  0.],
         [ 4.,  5.,  6.,  7.,  0.,  0.,  0.,  0.],
         [ 8.,  9., 10., 11.,  0.,  0.,  0.,  0.]]))

In [43]:
x == y

tensor([[ True, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])

### broadcasting mechanism

In [48]:
x = torch.arange(2, dtype = torch.float32).reshape((1,2))
y = torch.zeros(3, dtype = torch.float32).reshape((1,3))
x + y

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [31]:
x.sum()

tensor(66.)

In [51]:
x = torch.arange(12, dtype = torch.float32).reshape((3,4))
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [56]:
x[1:,:3]

tensor([[ 4.,  5.,  6.],
        [ 8.,  9., 10.]])

In [59]:
x[2,2]=13
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 13., 11.]])

### NumPy张量


In [62]:
A = x.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

### 线性代数

In [66]:
#标量可以用一个Tensor计算
a = torch.tensor([3.0])
b = torch.tensor([2.0])
a + b ,a-b, a*b, a/b

(tensor([5.]), tensor([1.]), tensor([6.]), tensor([1.5000]))

In [74]:
#矩阵的转置
a = torch.arange(12).reshape(3,4)
a, a.T

(tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[ 0,  4,  8],
         [ 1,  5,  9],
         [ 2,  6, 10],
         [ 3,  7, 11]]))

In [84]:
A = torch.arange(24).reshape(2,3,4)
B = A.clone() # 通过分配新内存， 将A的一个副本分配B
A, A + B,A * B

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],
 
         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]),
 tensor([[[ 0,  2,  4,  6],
          [ 8, 10, 12, 14],
          [16, 18, 20, 22]],
 
         [[24, 26, 28, 30],
          [32, 34, 36, 38],
          [40, 42, 44, 46]]]),
 tensor([[[  0,   1,   4,   9],
          [ 16,  25,  36,  49],
          [ 64,  81, 100, 121]],
 
         [[144, 169, 196, 225],
          [256, 289, 324, 361],
          [400, 441, 484, 529]]]))

In [105]:
 X = torch.arange(20).reshape(5,4)
 X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

In [88]:
X.shape

torch.Size([5, 4])

In [93]:
X.sum(axis = 0).shape

torch.Size([4])

In [106]:
B = X.sum(axis = 1,  keepdims=True).shape

In [110]:
 X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

### 自动求导 y = (2X^T) X, 对列向量X求导

In [111]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [115]:
x.requires_grad_(True) # 等价于x = torch.arange(4.0 requires_grad = True)
x.grad                 #  默认值是None

In [116]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [117]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [118]:
x.grad == 4*x

tensor([True, True, True, True])

 ### 另外一个函数

In [122]:
# 在默认情况下，PyTorch会累计梯度，我们需要清楚之前的值
x.grad.zero_()
y = x.sum()
y.backward()  
x.grad

tensor([1., 1., 1., 1.])

### 将某些计算移动到记录的计算图之外

## 1 线性回归的从零开始实现

In [2]:
%matplotlib inline
import random
import torch
from torch.utils import data
#from d2l import torch as d2l

In [8]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1,1))
 
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [9]:
print(features[0], "\n", labels[0])

tensor([-0.4393,  0.3507]) 
 tensor([2.1372])


In [11]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    #随机选取样本参与计算shuffle
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-1.4864,  0.9533],
        [ 0.1730,  0.4024],
        [ 0.0273,  0.3691],
        [-1.0225,  0.7568],
        [-1.2695,  1.0120],
        [-1.1528,  1.3344],
        [-0.5711,  0.2695],
        [-0.5230,  0.2817],
        [ 0.9252, -0.4066],
        [ 1.6433, -0.9494]]) 
 tensor([[-2.0152],
        [ 3.1816],
        [ 3.0035],
        [-0.4166],
        [-1.7907],
        [-2.6405],
        [ 2.1486],
        [ 2.1795],
        [ 7.4281],
        [10.6976]])


### 初始化模型参数

In [13]:
w = torch.normal(0,1, size=(2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

### 定义模型

In [14]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

### 定义损失函数

In [15]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2/2

### 定义优化算法

In [16]:
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad/batch_size
            param.grad.zero_()

### 训练过程

In [21]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')


epoch 1, loss 0.031348
epoch 2, loss 0.000127
epoch 3, loss 0.000048


In [22]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0006, -0.0005], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0002], grad_fn=<RsubBackward1>)


## 线性回归简洁实现

# SoftMax回归

In [30]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
from torchvision import transforms


In [42]:
# 通过ToTensor
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)

In [43]:
len(mnist_train), len(mnist_test)

(60000, 10000)

In [47]:
mnist_train[0][0].shape

torch.Size([1, 28, 28])

In [52]:
def get_fashion_mnist_labels(labels):  #@save
    """返回Fashion-MNIST数据集的文本标签"""
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save
    """绘制图像列表"""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # 图片张量
            ax.imshow(img.numpy())
        else:
            # PIL图片
            ax.imshow(img)
        ax.axes.get_xaxis() NB  .set_ visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

SyntaxError: invalid syntax (Temp/ipykernel_12748/3132057855.py, line 19)

In [51]:
X, y = next(iter(data.DataLoader(mnist_train, batch_size=18)))

### SoftMax实现分类 

In [53]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim = True)
    return X_exp/partition 
    

In [55]:
X = torch.normal(0,1,(2,5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0231, 0.2343, 0.6600, 0.0463, 0.0363],
         [0.2909, 0.3459, 0.0910, 0.2216, 0.0506]]),
 tensor([1.0000, 1.0000]))